# Proyecto ABC

In [18]:
import numpy as np
import screed 
from collections import defaultdict

In [19]:
# Invertible Hash function
def InvertibleHash(x, p):
    m = (2 ** p) - 1
    x = ((~x) + (x << 21)) & m
    x = x ^ (x >> 24)
    x = (x + (x << 3) + (x << 8)) & m
    x = x ^ (x >> 14)
    x = (x + (x << 2) + (x << 4)) & m
    x = x ^ (x >> 28)
    x = (x + (x << 31)) & m
    return x

# Natural hash
def NaturalHash(kmer, k):
    values =  {'A': 0, 'C': 1, 'G': 2, 'T': 3}
    x = 0
    for i in range(k):
        x += values[kmer[k - 1 - i]]  * (4 ** i)
    return x

# Composition of string and integer hash to avoid errors with Poly-A's
def Phi(kmer, k):
    return InvertibleHash(NaturalHash(kmer, k), 2 * k)

In [20]:
# Compute minimizers

# def MinimizerSketch(s, w, k):
#     M = set()
#     for i in range(len(s) - w - k + 1):
#         m = np.Inf
#         for j in range(w):
#             kmer = s[i + j: i + j + k]
#             rckmer = screed.rc(kmer)
#             u = Phi(kmer, k)
#             v = Phi(rckmer, k)
#             if u != v: 
#                 m = min(m, min(u, v))

#         for j in range(w):
#             kmer = s[i + j: i + j + k]
#             rckmer = screed.rc(kmer)
#             u = Phi(kmer, k)
#             v = Phi(rckmer, k)
#             if u < v and u == m:
#                 M.add((m, i + j, 0))
#             elif v < u and v == m:
#                 M.add((m, i + j, 1))
#     return M

def LocalMinimizers(queue):
    i = 1
    ranked = sorted(queue)
    t = ranked[0][0]
    if t < np.Inf:
        for M in ranked[1:]:
            if M[0] == t: 
                i += 1
            else:
                break 
        return ranked[:i]
    return []


def MinimizerSketch(s, w, k): 
    queue = [] 
    M = []
    for i in range(w):
        kmer = s[i: i + k]
        rckmer = screed.rc(kmer)
        u = Phi(kmer, k)
        v = Phi(rckmer, k)
        if u < v:
            queue.append((u, i, 0))
        if u == v: 
            queue.append((np.Inf, -1, -1))
        if u > v:
            queue.append((v, i, 1))
    M.extend(LocalMinimizers(queue))


    for i in range(w, len(s) - k + 1):
        kmer = s[i: i + k]
        rckmer = screed.rc(kmer)
        u = Phi(kmer, k)
        v = Phi(rckmer, k)
        if u < v:
            queue.append((u, i, 0))
        if u == v: 
            queue.append((np.Inf, -1, -1))
        if u > v:
            queue.append((v, i, 1))
        
        queue.pop(0)

        lastm = M[-1][0]
        lasti = M[-1][1]
        m = queue[-1][0]
        i = queue[-1][1]
        if m <= lastm:
            M.append(queue[-1])
        elif i - lasti >= w:
            M.extend(LocalMinimizers(queue))
    
    return M

In [21]:
# Index target sequences
def Index(T, w, k):
    A = []
    for t in range(len(T)):
        M = MinimizerSketch(T[t], w, k)
        for minimizer in M:
            h, i, r = minimizer
            seqminimizer = (h, t, i, r)
            A.append(seqminimizer)
    A.sort()
    H = defaultdict(list)
    for a in A:
        H[a[0]] = []
    for a in A:
        H[a[0]].append((a[1], a[2], a[3])) 
    return H

In [22]:
# Map a query sequence

def Map(H, q, w, k, epsilon):
    A = []
    M = MinimizerSketch(q, w, k)
    for minimizer in M:
        h, i, r = minimizer
        if h in H.keys():
            for hminimizer in H[h]:
                t, i_h, r_h = hminimizer
                if r == r_h:
                    A.append((t, 0, i - i_h, i_h)) 
                else:
                    A.append((t, 1, i + i_h, i_h))
    
    A.sort()
    b = 0
    maxchain = []
    for e in range(len(A)):
        if e == len(A) - 1 or A[e + 1][0] != A[e][0] or (
            A[e + 1][1] != A[e][1] or A[e + 1][2] - A[e][2] >= epsilon):
            chain = A[b: (e + 1)]
            if len(chain) >= 4 and len(chain) > len(maxchain):
                    maxchain = chain
            b = e + 1
    return maxchain


In [23]:
w = 4
k = 8
import time
t0 = time.time()
MinimizerSketch('AAATCCTGCTACCACATCGCCAGACACCA', w, k)
time.time() - t0

0.0006930828094482422

In [24]:
T = ['AAATCCTGCTACCACATCGCCAGACACCACAACCGACAACGACGAGATTGATGACAGCGCTGCGGCACGG', 
    'AAATCCTGCTACCACATCGCCAGTCACCACAACCGACAACGACGAGATTGATGACAGCGCTGCGGCACGG',
    screed.rc('AAATCCTGCTACCACATCGCCAGACACCACAACCGACAACGACGAGATTGATGACAGCGCTGCGGCACGG')]
w = 5
k = 15

H = Index(T, w, k)
q = 'CATCGCCAGACACCACAACCGACAACGACGAGATTGAT'
epsilon = 500
Map(H, q, w, k, epsilon)


[(0, 0, -14, 17),
 (0, 0, -14, 20),
 (0, 0, -14, 25),
 (0, 0, -14, 28),
 (0, 0, -14, 32),
 (0, 0, -14, 36)]

In [25]:
T = ['AAATCCTGCTACCACATCGCCAGACACCACAACCGACAACGACGAGATTGATGACAGCGCTGCGGCACGG', 
    'AAATCCTGCTACCACATCGCCAGTCACCACAACCGACAACGACGAGATTGATGACAGCGCTGCGGCACGG',
    screed.rc('AAATCCTGCTACCACATCGCCAGACACCACAACCGACAACGACGAGATTGATGACAGCGCTGCGGCACGG')]
w = 3
k = 10

H = Index(T, w, k)
q = 'CATCGCCAGACACCACAACCGACAACGACGAGATTGAT'
epsilon = 500
Map(H, q, w, k, epsilon)


[(0, 0, -14, 14),
 (0, 0, -14, 15),
 (0, 0, -14, 16),
 (0, 0, -14, 18),
 (0, 0, -14, 20),
 (0, 0, -14, 21),
 (0, 0, -14, 22),
 (0, 0, -14, 23),
 (0, 0, -14, 24),
 (0, 0, -14, 27),
 (0, 0, -14, 28),
 (0, 0, -14, 30),
 (0, 0, -14, 32),
 (0, 0, -14, 33),
 (0, 0, -14, 36),
 (0, 0, -14, 38),
 (0, 0, -14, 40)]

In [26]:
def GetSequencesFromFile(fileName):
    sequences = []
    for record in screed.open(fileName):
        sequences.append(record.sequence)
    return sequences

In [31]:
mtuberculosis = GetSequencesFromFile('Mtuberculosis.fasta')
test = [mtuberculosis[0]]
w = 5
k = 15


H = Index(test, w, k)


In [36]:
# Para formatear el resultado en un archivo SAM dummy
i = 0
with open('test.sam', 'w') as f:
    for record in screed.open('SRR8186772_subsample_16k.fasta'):
        i += 1
        print('tiempo: ', i)
        Seq = record.sequence
        maxchain = Map(H, Seq, w, k, epsilon)
        Seq_ID = record.name
        if not maxchain:
            flag = 4
        elif maxchain[0][1] == 0:
            flag = 0
        else:
            flag = 16
        if maxchain:
            pos = maxchain[0][3] + 1
        else: 
            pos = 0
        # Seq_ID = "El identificador (encabezado) de la secuencia, los que en el archivo con las secuencias empeiza con >. También podemos poner cualquier cosa por ahora"
        # flag = "0 si mapea, 4 si no mapea, 16 si mapea pero en el reverso complemento (este es opcional, puede ser 0)."
        # Seq = "La secuencia como tal. También podríamos poner cualqueir secuencia tipo AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA."
        f.write(f'{Seq_ID}\t{flag}\t*\t{pos}\t0\t*\t*\t0\t0\t{Seq}\t*\n')
    

tiempo:  1
tiempo:  2
tiempo:  3
tiempo:  4
tiempo:  5
tiempo:  6
tiempo:  7
tiempo:  8
tiempo:  9
tiempo:  10
tiempo:  11
tiempo:  12
tiempo:  13
tiempo:  14
tiempo:  15
tiempo:  16
tiempo:  17
tiempo:  18
tiempo:  19
tiempo:  20
tiempo:  21
tiempo:  22
tiempo:  23
tiempo:  24
tiempo:  25
tiempo:  26
tiempo:  27
tiempo:  28
tiempo:  29
tiempo:  30
tiempo:  31
tiempo:  32
tiempo:  33
tiempo:  34
tiempo:  35
tiempo:  36
tiempo:  37
tiempo:  38
tiempo:  39
tiempo:  40
tiempo:  41
tiempo:  42
tiempo:  43
tiempo:  44
tiempo:  45
tiempo:  46
tiempo:  47
tiempo:  48
tiempo:  49
tiempo:  50
tiempo:  51
tiempo:  52
tiempo:  53
tiempo:  54
tiempo:  55
tiempo:  56
tiempo:  57
tiempo:  58
tiempo:  59
tiempo:  60
tiempo:  61
tiempo:  62
tiempo:  63
tiempo:  64
tiempo:  65
tiempo:  66
tiempo:  67
tiempo:  68
tiempo:  69
tiempo:  70
tiempo:  71
tiempo:  72
tiempo:  73
tiempo:  74
tiempo:  75
tiempo:  76
tiempo:  77
tiempo:  78
tiempo:  79
tiempo:  80
tiempo:  81
tiempo:  82
tiempo:  83
tiempo:  84
t

True
